<a href="https://colab.research.google.com/github/krazyguy/book-recommendation-knn/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-01-16 15:02:44--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   143MB/s    in 0.2s    

2025-01-16 15:02:44 (143 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your cdf_booksode here - consider creating a new cell for each section of code

In [5]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [6]:
df_ratings.user.value_counts()
# Get the count of ratings per user
user_rating_counts = df_ratings['user'].value_counts()

# Filter out users who have rated fewer than 200 books
users_to_drop = user_rating_counts[user_rating_counts < 200].index

books_rating_counts = df_ratings['isbn'].value_counts()

# Filter out users who have rated fewer than 200 books
books_to_drop = books_rating_counts[books_rating_counts < 100].index


# Drop the users and books that don't meet the criteria
df_ratings_filtered = df_ratings[~df_ratings['user'].isin(users_to_drop)]
df_ratings_filtered = df_ratings_filtered[~df_ratings_filtered['isbn'].isin(books_to_drop)]


df_ratings_filtered.shape

(49781, 3)

In [7]:
df_books.describe()

,isbn,title,author
count,271379,271379,271377
unique,271379,242154,102041
top,0195153448,Selected Poems,Agatha Christie
freq,1,27,632


In [8]:
merged_df = pd.merge(df_ratings_filtered, df_books, on='isbn')
merged_df.head()

,user,isbn,rating,title,author
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
2,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
3,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
4,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff


In [9]:
merged_df=merged_df.drop_duplicates(['user','title'])

In [10]:
merged_df.shape

(49136, 5)

In [11]:
user_item_matrix = merged_df.pivot_table(
    index='title',
    columns='user',
    values='rating'
)
user_item_matrix = user_item_matrix.fillna(0)
user_item_matrix.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
print(user_item_matrix.shape)

(673, 888)


In [13]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

user_item_matrix_values=user_item_matrix.values
knn = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')
knn.fit(user_item_matrix_values)


NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [14]:
def get_recommends(book_title):
    try:
        # Find the index of the book in the pivot table
        book_index = user_item_matrix.index.get_loc(book_title)

        # Get the ratings (vector) for the book
        book_vector = user_item_matrix.iloc[book_index, :].values.reshape(1, -1)

        # Find the 6 nearest neighbors (including the book itself)
        distances, indices = knn.kneighbors(book_vector, n_neighbors=6)

        # Initialize an empty list to store the recommended books
        recommended_books = []

        # Iterate over the distances and indices of the nearest neighbors
        for distance, index in zip(distances[0], indices[0]):
            # Exclude the book itself (distance == 0)
            if distance != 0:
                # Get the book title using the index
                similar_book_title = user_item_matrix.index[index]

                # Append the book title and distance to the list
                recommended_books.append([similar_book_title, distance])

        # Format the recommendations: the original book title followed by similar books
        return [book_title, recommended_books[::-1]]  # Reverse the order to show closest first

    except KeyError:
        # Return if the book is not found in the dataset
        return [book_title, "Book not found in dataset."]


In [15]:
recommendations = get_recommends("Catch 22")
print(recommendations)


['Catch 22', [['Last Man Standing', 0.74926436], ['How the Garcia Girls Lost Their Accents (Plume Contemporary Fiction)', 0.74907064], ['The Bone Collector (Lincoln Rhyme Novels (Paperback))', 0.73021567], ['The World According to Garp', 0.728256], ['The Witching Hour (Lives of the Mayfair Witches)', 0.706072]]]


In [16]:

recommendations = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(recommendations)

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['Catch 22', 0.7939835], ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566], ['Interview with the Vampire', 0.73450685], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]


In [17]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
